# Sqlalcmemy 1.4/2.0 Tutorial

## Establishing Connectity - the Engine

In [1]:
from sqlalchemy import create_engine
engine = create_engine(
    "sqlite+pysqlite:///:memory:", 
    echo=True, future=True)

## Working with Transactions and the DBAPI

### Getting a Connection

In [2]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("select 'hello world'"))
    print(result.all())

2022-04-03 17:57:33,506 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:33,507 INFO sqlalchemy.engine.Engine select 'hello world'
2022-04-03 17:57:33,508 INFO sqlalchemy.engine.Engine [generated in 0.00170s] ()
[('hello world',)]
2022-04-03 17:57:33,509 INFO sqlalchemy.engine.Engine ROLLBACK


### Committing Changes

In [3]:
with engine.connect() as conn:
    conn.execute(
        text("CREATE TABLE "
        "some_table "
        "(x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table "
        "(x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, 
        {"x": 2, "y": 4}]
    )
    conn.commit()


2022-04-03 17:57:33,624 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:33,625 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-04-03 17:57:33,626 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ()
2022-04-03 17:57:33,627 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-03 17:57:33,628 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ((1, 1), (2, 4))
2022-04-03 17:57:33,629 INFO sqlalchemy.engine.Engine COMMIT


The "begin once" style

In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO"
        " some_table (x, y)"
        " VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, 
        {"x": 9, "y": 10}]
    )

2022-04-03 17:57:33,741 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:33,743 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-03 17:57:33,743 INFO sqlalchemy.engine.Engine [cached since 0.1162s ago] ((6, 8), (9, 10))
2022-04-03 17:57:33,744 INFO sqlalchemy.engine.Engine COMMIT


### Basics of Statement Execution

#### Fetching Rows

In [5]:
with engine.connect() as conn:
    result = conn.execute(
        text(
            "SELECT x, y "
            "FROM some_table"))
    for row in result:
        print(f"x: {row.x} "
        f" y: {row.y}")

2022-04-03 17:57:33,858 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:33,859 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2022-04-03 17:57:33,860 INFO sqlalchemy.engine.Engine [generated in 0.00155s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-04-03 17:57:33,861 INFO sqlalchemy.engine.Engine ROLLBACK


#### Sending Parameters

In [6]:
with engine.connect() as conn:
    result = conn.execute(
        text("SELECT x, y FROM some_table WHERE y > :y"),
        {"y": 2}
    )
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-03 17:57:33,981 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:33,982 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2022-04-03 17:57:33,983 INFO sqlalchemy.engine.Engine [generated in 0.00227s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-04-03 17:57:33,985 INFO sqlalchemy.engine.Engine ROLLBACK


#### Sending Multiple Parameters

In [7]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}]
    )
    conn.commit()


2022-04-03 17:57:34,112 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:34,113 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-03 17:57:34,114 INFO sqlalchemy.engine.Engine [cached since 0.4872s ago] ((11, 12), (13, 14))
2022-04-03 17:57:34,116 INFO sqlalchemy.engine.Engine COMMIT


#### Bunding Parameters with a Statement

In [8]:
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-03 17:57:34,222 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:34,223 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-04-03 17:57:34,224 INFO sqlalchemy.engine.Engine [generated in 0.00160s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-04-03 17:57:34,224 INFO sqlalchemy.engine.Engine ROLLBACK


#### Executing with an ORM Session

In [9]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-03 17:57:34,408 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:34,409 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-04-03 17:57:34,410 INFO sqlalchemy.engine.Engine [cached since 0.1881s ago] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-04-03 17:57:34,410 INFO sqlalchemy.engine.Engine ROLLBACK


commit as you go

In [10]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y":11}, {"x": 13, "y": 15}]
    )
    session.commit()


2022-04-03 17:57:34,513 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:34,514 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2022-04-03 17:57:34,515 INFO sqlalchemy.engine.Engine [generated in 0.00065s] ((11, 9), (15, 13))
2022-04-03 17:57:34,517 INFO sqlalchemy.engine.Engine COMMIT


## Working with Database Metadata

### Setting up MetaData with Table objects

In [11]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [12]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)

Column

In [13]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [14]:
user_table.c.keys()

['id', 'name', 'fullname']

### Declaring Simple Constraints

In [15]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [16]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, 
    primary_key=True),
    Column('user_id',
     ForeignKey('user_account.id'), 
     nullable=False),
    Column('email_address',
     String,
     nullable=False)
)

### Emitting DDL to the Database

In [17]:
metadata_obj.create_all(engine)

2022-04-03 17:57:35,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:35,352 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-03 17:57:35,353 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:35,355 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-04-03 17:57:35,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:35,359 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-03 17:57:35,360 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:35,361 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-04-03 17:57:35,362 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:35,363 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-04-03 17:57:35,363 INFO sqlalchemy.engine.Engine [no key 0.00052s] ()
2022-04-03 17:57:35,365 INFO sqlalchemy.engine.Engine 
C

### Defining Table Metadata with the ORM

#### Setting up the Registry

In [18]:
from sqlalchemy.orm import registry
mapper_registry = registry()

In [19]:
mapper_registry.metadata

MetaData()

In [20]:
Base = mapper_registry.\
    generate_base()

#### Declaring Mapped Classes

In [21]:
from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = 'user_account'
    id = Column(Integer, 
    primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address",
     back_populates="user")
    def __repr__(self):
       return f"User(id={self.id!r}," \
       f" name={self.name!r}," \
       f" fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, 
    primary_key=True)
    email_address = Column(String,
     nullable=False)
    user_id = Column(Integer, 
    ForeignKey('user_account.id'))
    user = relationship("User", 
    back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r},"\
        f" email_address="\
        f"{self.email_address!r})"

In [22]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

#### Other Mapped Class Details

the classes have an automatically generated `__init__()` method

In [23]:
sandy = User(name='sandy',
 fullname='Sandy Cheeks')

we provided a `__repr__()` method

In [24]:
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

we also included a bidirectional relationship

#### Emitting DDL to the database

In [25]:
# emit CREATE statements given ORM registry
mapper_registry.metadata.create_all(engine)

# the identical MetaData object is also present on the
# declarative base
Base.metadata.create_all(engine)

2022-04-03 17:57:36,481 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:36,483 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-03 17:57:36,483 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:36,485 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-03 17:57:36,486 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:36,487 INFO sqlalchemy.engine.Engine COMMIT
2022-04-03 17:57:36,491 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:36,492 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-03 17:57:36,493 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:36,494 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-03 17:57:36,495 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:36,496 INFO sqlalchemy.engine.Engine COMMIT


#### Combining Core Table Declarations with ORM Declarative

In [26]:
# class User(Base):
#     __table__ = user_table

#     addresses = relationship("Address", back_populates="user")

#     def __repr__(self):
#         return f"User({self.name!r}, {self.fullname!r})"

# class Address(Base):
#     __table__ = address_table

#     user = relationship("User", back_populates="addresses")

#     def __repr__(self):
#         return f"Address({self.email_address!r})"

### Table Reflection

In [27]:
some_table = Table("some_table",
 metadata_obj, autoload_with=engine)


2022-04-03 17:57:36,693 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:36,694 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-04-03 17:57:36,695 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:36,696 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-03 17:57:36,697 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-04-03 17:57:36,699 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-04-03 17:57:36,700 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:36,701 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-04-03 17:57:36,702 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:57:36,703 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
20

In [28]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

## Working With Data

### Inserting Rows with Core

The insert() SQL Expression Construct

In [29]:
from sqlalchemy import insert
stmt = insert(user_table).\
    values(name='spongebob', 
    fullname="Spongebob Squarepants")

In [30]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [31]:
compiled = stmt.compile()

In [32]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

Executing the Statement

In [33]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()


2022-04-03 17:57:37,668 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:37,670 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-03 17:57:37,672 INFO sqlalchemy.engine.Engine [generated in 0.00492s] ('spongebob', 'Spongebob Squarepants')
2022-04-03 17:57:37,675 INFO sqlalchemy.engine.Engine COMMIT


In [34]:
result.inserted_primary_key

(1,)

INSERT usually generates the "values" clause automatically

In [35]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", 
            "fullname": "Sandy Cheeks"},
            {"name": "patrick", 
            "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-04-03 17:57:37,957 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:37,959 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-03 17:57:37,959 INFO sqlalchemy.engine.Engine [generated in 0.00201s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-04-03 17:57:37,960 INFO sqlalchemy.engine.Engine COMMIT


Deep Alchemy

In [36]:
from sqlalchemy import select, bindparam

scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": "spongebob", "email_address": "spongebob@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()


2022-04-03 17:57:38,079 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:38,080 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-04-03 17:57:38,080 INFO sqlalchemy.engine.Engine [generated in 0.00166s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-04-03 17:57:38,082 INFO sqlalchemy.engine.Engine COMMIT


INSERT...FROM SELECT

In [37]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)


INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


INSERT...RETURNING

In [38]:
insert_stmt = insert(address_table).returning(address_table.c.id, address_table.c.email_address)
print(insert_stmt)


INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [39]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))


INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


## Selecting Rows with Core or ORM

### The select() SQL Expression Construct

In [40]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == 'spongebob')
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [41]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-04-03 17:57:38,795 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:57:38,796 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-03 17:57:38,796 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-04-03 17:57:38,798 INFO sqlalchemy.engine.Engine ROLLBACK


In [42]:
# class User(Base):
#     __table__ = user_table

#     addresses = relationship("Address", back_populates="user")

#     def __repr__(self):
#         return f"User({self.name!r}, {self.fullname!r})"

# class Address(Base):
#     __table__ = address_table

#     user = relationship("User", back_populates="addresses")

#     def __repr__(self):
#         return f"Address({self.email_address!r})"

In [44]:
stmt = select(User).where(User.name == 'spongebob')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-04-03 18:47:29,725 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:47:29,726 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-03 18:47:29,726 INFO sqlalchemy.engine.Engine [cached since 2988s ago] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-04-03 18:47:29,728 INFO sqlalchemy.engine.Engine ROLLBACK
